## Assymetric Cryptography

In [216]:
import random
import math
import codecs
import base64

In [253]:
class Utils:
    def __init__(self):
        '''if not used then delete'''
    
    def hex_to_dec(hex):
        deci = int(hex, 16)
        return deci

    def dec_to_hex(dec):
        hexa = hex(dec)
        return hexa

    def hex_to_b64(hex):
        hex = hex[2:] if(len(hex[2:]) % 2 == 0) else '0' + hex[2:]
        b64 = codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()
        return b64
       
    def b64_to_hex(b64):
        hexa = base64.b64decode(b64).hex()
        return hexa
    
    def dec_to_b64(dec):
        b64 = Utils.hex_to_b64(Utils.dec_to_hex(dec))
        return b64

    def b64_to_dec(b64):
        deci = Utils.hex_to_dec(Utils.b64_to_hex(b64))
        return deci

    def string_to_b64(string):
        b64= base64.b64encode(string.encode("ascii")).decode("ascii")
        return b64

    def b64_to_string(b64):
        string = base64.b64decode(b64.encode("ascii")).decode("ascii")
        return string

    def is_square(i):
        return i == math.isqrt(i) ** 2

    def mod_sqrt(n, p):

        points = []
        n = n % p
        for x in range (1, p):
            if (pow(x, 2, p) == n):
                points.append(x)
        return points

        

In [259]:

class ElGamal:
    def __init__(self):
        pass

    def create_key(self, p, g, x):
        y = pow(g, x, p)
        public_key = { 'y': y, 'g': g, 'p': p }
        private_key = { 'x': x, 'p': p }
        return public_key, private_key

    def encrypt(self, message, key, public_key):
        a = pow(public_key['g'], key, public_key['p'])
        b = (pow(public_key['y'], key, public_key['p']) * (message % public_key['p'])) % public_key['p']
        return { 'a': a, 'b': b }

    def decrypt(self, message, private_key):
        return ((message['b'] % private_key['p']) * pow(message['a'], (private_key['p'] - 1 - private_key['x']), private_key['p'])) % private_key['p']


In [260]:
elgamal = ElGamal()

public_key, private_key = elgamal.create_key(2357, 2, 1751)

print("Public Key\t: " + str(public_key))
print("Private Key\t: " + str(private_key))

encrypted = elgamal.encrypt(2035, 1520, public_key)
print("Ciphertext\t: " + str(encrypted))

decrypted = elgamal.decrypt(encrypted, private_key)
print("Plaintext\t: " + str(decrypted))

Public Key	: {'y': 1185, 'g': 2, 'p': 2357}
Private Key	: {'x': 1751, 'p': 2357}
Ciphertext	: {'a': 1430, 'b': 697}
Plaintext	: 2035


In [263]:
class ElGamalMachine:
    def __init__(self):
        self.elgamal = ElGamal()
    
    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_key(self, bit):
        p = self.create_random_prime(bit)
        g = random.randint(1, p - 1)
        x = random.randint(1, p -2)

        public_key, private_key = self.elgamal.create_key(p, g, x)

        public_key['y'] = Utils.dec_to_b64(public_key['y'])
        public_key['g'] = Utils.dec_to_b64(public_key['g'])
        public_key['p'] = Utils.dec_to_b64(public_key['p'])

        private_key['x'] = Utils.dec_to_b64(private_key['x'])
        private_key['p'] = Utils.dec_to_b64(private_key['p'])

        pub_key = public_key['y'] + public_key['g'] + public_key['p']
        pri_key = private_key['x'] + private_key['p']

        return pub_key, pri_key

    def create_key_file(self, bit):
        public_key, private_key = self.create_key(bit)

        pub = open("key.pub", "w")
        pub.write(public_key)

        pri = open("key.pri", "w")
        pri.write(private_key)
        

    def encrypt(self, message, public_key):
        y = Utils.b64_to_dec(public_key['y'])
        g = Utils.b64_to_dec(public_key['g'])
        p = Utils.b64_to_dec(public_key['p'])

        p_key = { 'y': y, 'g': g, 'p': p }

        encrypted = ''
        for m in message:
            k = random.randint(1, p-2)
            res = self.elgamal.encrypt(ord(m), k, p_key)
            a = Utils.dec_to_b64(res['a'])
            b = Utils.dec_to_b64(res['b'])
            encrypted += a + b
        return encrypted

    def encrypt_full(self, message, pub_key):
        public_key = pub_key.split("=\n")[:-1]
        public_key = { 'y': public_key[0] + "=\n", 'g': public_key[1] + "=\n", 'p': public_key[2] + "=\n" }

        encrypted = self.encrypt(message, public_key)
        return encrypted

    def decrypt_full(self, encrypted, pri_key):
        private_key = pri_key.split("=\n")[:-1]
        private_key = { 'x': private_key[0] + "=\n", 'p': private_key[1] + "=\n" }
        
        decrypted = machine.decrypt(encrypted, private_key)
        return decrypted


    def decrypt(self, encrypted, private_key):
        x = Utils.b64_to_dec(private_key['x'])
        p = Utils.b64_to_dec(private_key['p'])

        p_key = { 'x': x, 'p': p }
        
        encrypted = encrypted.split('=\n')[:-1]
        encrypted_paired = []
        for i in range(0, len(encrypted), 2):
            a = Utils.b64_to_dec(encrypted[i] + '=\n')
            b = Utils.b64_to_dec(encrypted[i+1] + '=\n')
            encrypted_paired.append({ 'a': a, 'b': b })

        message = ''
        for m in encrypted_paired:
            message += chr(self.elgamal.decrypt(m, p_key))
        return message

In [265]:
machine = ElGamalMachine()

public_key, private_key = machine.create_key(512)

print(public_key)
print(private_key)

# machine.create_key_file(256)

H3VwtRq4KcCxe3t2AnOGQkxY/pasmqbVuEUBN8rkMu2Uca6KMWmbhLQqcUYk78aH05zdV/iu6KT3
aIvdY2/mKg==
OCyXuA9XnmlSyQSJQio/5kw9puwV8Lmc366JLH0EeflqbaWdbi2EIy1Ma7Xukonql7iximcFu1ou
eSBpIDo9vw==
3RY1XQHy4TCG1s2/hxnba9KofVYytuFcEKq0mOfkT7yq2rmqhDBTr9vZdj3xDjtU9QEDwOA16NPZ
Vk++KdgqtQ==

nZTchKMeTkg3h4Q+3MSmrAivb+w5y52irVnr5C8d2As+2SiNT2ef7mgjJKS13JBL0ZJOS6BQaSp2
lcaZ/dhsfg==
3RY1XQHy4TCG1s2/hxnba9KofVYytuFcEKq0mOfkT7yq2rmqhDBTr9vZdj3xDjtU9QEDwOA16NPZ
Vk++KdgqtQ==



In [268]:
machine.encrypt_full("AUFA F", public_key)

'n3boPrtN735+H4HuyU/lQFyAH4wSL1T/jph4WB/3vIzfQQrLSq2eeAP5G4Eo7DijvDov+J+jZOJ2\nKP4GtOVmkg==\nmWgRQAK6jgkIcVp9flBNejc7Go0qFs8uJ1sf9PLE289S9R0Y/luBNb3P0KoCSuThTqv4wALs8nHE\nuD4HMIYcBA==\nr/vICe21rrUUoizOIyuwUWs81xL2k5h0dGod9lsThL2eSb/hmNMyD8oU5wLIB5XROFZAoeciD0RZ\nMw5UHXGZog==\nzSsPYvRoPlWXp8XXOjgdSFJZ8yNBJAINH7docjBhaPEEnplAtO+5xX+j5qMQVvoPIj7SsZnHAknf\n6t9lru+0pg==\njtpzr93WTBfQr559nlW0UyKOxNZVBWLi9UaFh9KhzHsFdvO7chUzm9qjR0BochlA9bjzs/+V6a7Q\nkNQ8dklL0Q==\noZ9O9BxSqk0EGn3XjJ0oUAUdRtAraz53EqisyNgxKV8WeImZATWBvLENIDiJtjztYID6XF5EYGLk\nowrPq/D35A==\nRlIv4P0aFMB2rg8reZDs0HMlpDKMhbTdYEBeEVxwcQTD7pAs7V6MopjIOX42Nr8BWHOmSWsv/BEw\ndWLpoIs+Mw==\nZgqhjuX63vfDK50l5H3g8anjHQY0Bw12jw3OP2UdvYg2MTVFM+GxSpM2YtoTLJAXYLpkMF7Q6VWo\neIVKLOJtJA==\nFe6K9uWNJssedHzQ3ZMfsAm51Q0t31v7+GxE+RIxP7JstR6gCAFiOrOwqqa9C5GbP9Xgy4Y3GELV\naoStaQaQOg==\ne259LWD/xdDJjuRr8hu6GnaHjNelgdK7YL5GR7fiSza8DnJNocme9GYJ5Vw/QZqTCLM91FhUj/eY\n1LBrI8w56g==\nYX0HEMaDIfvkuOe5fq0c0LY27BkMF7LuWTe0ja2GkNmTHIse5BmFNesBQfiA9TmpZBXGNNfCzedt\no

In [269]:
machine.decrypt_full("n3boPrtN735+H4HuyU/lQFyAH4wSL1T/jph4WB/3vIzfQQrLSq2eeAP5G4Eo7DijvDov+J+jZOJ2\nKP4GtOVmkg==\nmWgRQAK6jgkIcVp9flBNejc7Go0qFs8uJ1sf9PLE289S9R0Y/luBNb3P0KoCSuThTqv4wALs8nHE\nuD4HMIYcBA==\nr/vICe21rrUUoizOIyuwUWs81xL2k5h0dGod9lsThL2eSb/hmNMyD8oU5wLIB5XROFZAoeciD0RZ\nMw5UHXGZog==\nzSsPYvRoPlWXp8XXOjgdSFJZ8yNBJAINH7docjBhaPEEnplAtO+5xX+j5qMQVvoPIj7SsZnHAknf\n6t9lru+0pg==\njtpzr93WTBfQr559nlW0UyKOxNZVBWLi9UaFh9KhzHsFdvO7chUzm9qjR0BochlA9bjzs/+V6a7Q\nkNQ8dklL0Q==\noZ9O9BxSqk0EGn3XjJ0oUAUdRtAraz53EqisyNgxKV8WeImZATWBvLENIDiJtjztYID6XF5EYGLk\nowrPq/D35A==\nRlIv4P0aFMB2rg8reZDs0HMlpDKMhbTdYEBeEVxwcQTD7pAs7V6MopjIOX42Nr8BWHOmSWsv/BEw\ndWLpoIs+Mw==\nZgqhjuX63vfDK50l5H3g8anjHQY0Bw12jw3OP2UdvYg2MTVFM+GxSpM2YtoTLJAXYLpkMF7Q6VWo\neIVKLOJtJA==\nFe6K9uWNJssedHzQ3ZMfsAm51Q0t31v7+GxE+RIxP7JstR6gCAFiOrOwqqa9C5GbP9Xgy4Y3GELV\naoStaQaQOg==\ne259LWD/xdDJjuRr8hu6GnaHjNelgdK7YL5GR7fiSza8DnJNocme9GYJ5Vw/QZqTCLM91FhUj/eY\n1LBrI8w56g==\nYX0HEMaDIfvkuOe5fq0c0LY27BkMF7LuWTe0ja2GkNmTHIse5BmFNesBQfiA9TmpZBXGNNfCzedt\noqVtjBBRTA==\nmiGdYn6PUFxZsfamURXXbF1JCTv9nOE4wR1Bh+3bh6Qwgrq1xkFQdG8QiEieSE948V1rQfypIafB\nqKz3L5cJ3A==\n", private_key)

'AUFA F'

In [126]:
class EllipticPoint:
    def __init__(self, x=0, y=0, inf=False):
        self.x = x if not inf else math.inf
        self.y = y if not inf else math.inf
        self.inf = inf

    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def is_infinity(self):
        return self.inf
    
    def mirror(self):
        return EllipticPoint(self.x, -self.y, self.inf)

    def __eq__(self, point):
        if(isinstance(point, EllipticPoint)):
            if(self.inf):
                return self.inf == point.inf
            else:
                return self.x == point.x and self.y == point.y
            
        return False

    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)

In [187]:
class EllipticCurve:
    def __init__(self, a, b, p):
        ''' y^2 = x^3 + ax + b '''
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        if(self.is_singular()):
            print("Error: Curve is Singular!.")
    
    def is_singular(self):
        return((((4 * (self.a ** 3)) + (27 * (self.b ** 2))) % self.p) == 0)

    def get_points(self):
        points = [EllipticPoint(inf=True)]
        
        for x in range(self.p):
            y_square = x ** 3 + self.a * x + self.b

            for point in Utils.mod_sqrt(y_square % self.p, self.p):
                points.append(EllipticPoint(x, point))
        self.points = points 
    
    def get_y_square(self, x):
        y_square = x ** 3 + self.a * x + self.b

        for point in Utils.mod_sqrt(y_square % self.p, self.p):
            return(EllipticPoint(x, point))

    def point_addition(self, P, Q):
        if(P == Q):
            return self.point_multiplication(P)
        else:
            xp = P.get_x(); yp = P.get_y();
            xq = Q.get_x(); yq = Q.get_y();

            if(xp == xq):
                return EllipticPoint(inf=True)
            else:
                m = ((yp-yq) * pow((xp-xq), -1, self.p)) % self.p

                xr = (m ** 2 - xp - xq) % self.p
                yr = (m * (xp - xr) - yp) % self.p
                return EllipticPoint(xr, yr)

    def point_substraction(self, P, Q):
        return self.point_addition(P, EllipticPoint(Q.get_x(), -Q.get_y() % self.p))

    def point_multiplication(self, P):
        
        xp = P.get_x(); yp = P.get_y();

        if(yp == 0):
            return EllipticPoint(inf=True)
        else:
            m = ((3 * (xp ** 2) + self.a) * pow((2 * yp), -1, self.p)) % self.p
            xr = (m ** 2 - 2 * xp) % self.p
            yr = (m * (xp - xr) - yp) % self.p
            return EllipticPoint(xr, yr)

    def point_scalar_multiplication(self, P, k):
        if(k == 1):
            return P
        elif(k == 2):
            return self.point_multiplication(P)
        else:
            prev = P
            res = self.point_multiplication(P)
            for i in range(k-2):
                if(res.is_infinity()):
                    res = self.point_addition(prev, self.point_multiplication(P))
                else:
                    prev = res
                    res = self.point_addition(res, P)
    
            return res
    

In [201]:
curve = EllipticCurve(1, 6, 11)
curve.get_points()


P = EllipticPoint(8, 8)

for i in range(1, len(curve.points)):
    print(curve.point_scalar_multiplication(P, i))



P = EllipticPoint(3, 5)
Q = EllipticPoint(5, 9)
print()
print(curve.point_addition(P, Q))


(8, 8)
(7, 2)
(10, 2)
(2, 7)
(5, 9)
(3, 5)
(3, 6)
(5, 2)
(2, 4)
(10, 9)
(7, 9)
(8, 3)

(7, 9)


In [256]:
class ECCElGamalMachine:
    def __init__(self):
        pass

    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_agreement(self, bit):
        p = self.create_random_prime(bit)

        curve_a = random.randint(-10, 10)
        curve_b = random.randint(-200, 200)

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        found = False

        B = None

        while(not found):
            B = curve.points[random.randint(1, len(curve.points) - 2)]
            if(not B.is_infinity()):
                found = True
        k = 2

        return p, curve_a, curve_b, B, k

    def create_key(self, p, curve_a, curve_b, B):

        pri = random.randint(1, p-2)

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()
       
        public_key = curve.point_scalar_multiplication(B, pri)
        private_key = pri

        return public_key, private_key

    def create_key_full(self, bit):
        p, curve_a, curve_b, B, k = self.create_agreement(bit)
        public_key, private_key = self.create_key(p, curve_a, curve_b, B)

        public_key = str(public_key.x) + "," + str(public_key.y)
        B = str(B.x) + "," + str(B.y)

        pub_key = public_key + " " + str(p) + " " + str(curve_a) + " " + str(curve_b) + " " + B + " " + str(k)
        pri_key = str(private_key) + " " + str(p) + " " + str(curve_a) + " " + str(curve_b) + " " + str(k)
        
        pub_key = Utils.string_to_b64(pub_key)
        pri_key = Utils.string_to_b64(pri_key)
        
        return pub_key, pri_key

    def encrypt_full(self, message, pub_key):
        pub_key = Utils.b64_to_string(pub_key)
        pub_key = pub_key.split(" ")

        public_key  = pub_key[0].split(",")
        public_key  = EllipticPoint(int(public_key[0]), int(public_key[1]))
        p           = int(pub_key[1])
        curve_a     = int(pub_key[2])
        curve_b     = int(pub_key[3])
        B           = pub_key[4].split(",")
        B           = EllipticPoint(int(B[0]), int(B[1]))
        k           = int(pub_key[5])

        encrypted = self.encrypt(message, public_key, p, curve_a, curve_b, B, k)

        encrypted_text = ""

        for i in range(len(encrypted)):
            e = encrypted[i]
            encrypted_text += str(e['kB'].x) + "," + str(e['kB'].y) + "," + str(e['PmkPb'].x) + "," + str(e['PmkPb'].y)
            if(i != len(encrypted)-1):
                encrypted_text += "\n"

        return encrypted_text

    def decrypt_full(self, encrypted_text, pri_key):
        
        encrypted = []
        encrypted_text = encrypted_text.split("\n")
        for i in range(len(encrypted_text)):
            e = encrypted_text[i].split(",")
            kB = EllipticPoint(int(e[0]), int(e[1]))
            PmkPb = EllipticPoint(int(e[2]), int(e[3]))
            encrypted.append({'kB': kB, 'PmkPb': PmkPb})


        pri_key = Utils.b64_to_string(pri_key)
        pri_key = pri_key.split(" ")

        private_key = int(pri_key[0])
        p           = int(pri_key[1])
        curve_a     = int(pri_key[2])
        curve_b     = int(pri_key[3])
        k           = int(pri_key[4])

        decrypted = self.decrypt(encrypted, private_key, p, curve_a, curve_b, k)

        return decrypted


    def point_available(self, points, x):
        res = None
        for point in points:
            if(point.get_x() == x):
                res = point
                break
        return res

    def encode(self, message, p, curve_a, curve_b, k):

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        encoded = []

        for m in message:
            m = ord(m)
            
            point = curve.points[k*m + 1]

            encoded.append(point)  
        return encoded

    def decode(self, encrypted, p, curve_a, curve_b, k):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()
        points = curve.points
        decoded = ""

        for e in encrypted:
            for i in range(len(points)):
                if(points[i].get_x() == e.get_x()):
                    break
            decoded += chr(math.floor((i - 1) / k))
        return decoded

    def encrypt(self, message, public_key, p, curve_a, curve_b, B, k):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        encoded = self.encode(message, p, curve_a, curve_b, k)

        print("ENCODED")
        for e in encoded:
            print(e)
        print()

        encrypted = []

        for encode in encoded:
            found = False
            while(not found):
                random_k = random.randint(1, p-1)
                kB = curve.point_scalar_multiplication(B, random_k)
                PmkPb = curve.point_addition(encode, curve.point_scalar_multiplication(public_key, random_k))
                if(not kB.is_infinity() and not PmkPb.is_infinity()):
                    found = True
            
            encrypted.append({ 'kB': kB, 'PmkPb': PmkPb })
            print("kB\t\t:" + str(kB))
            print("PmkPb\t:" + str(PmkPb))

        return encrypted

    def decrypt(self, encrypted, private_key, p, curve_a, curve_b, k):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        decrypted = []

        for e in encrypted:
            # print("PmkPb\t: " + str(e['PmkPb']))
            # print("Mul\t: " + str(curve.point_scalar_multiplication(e['kB'], private_key)))
            decrypted.append(curve.point_substraction(e['PmkPb'], curve.point_scalar_multiplication(e['kB'], private_key)))

        print("DECRYPTED")
        for e in decrypted:
            print(e)
        decoded = self.decode(decrypted, p, curve_a, curve_b, k)

        print()
        return decoded
            


In [257]:
machine = ECCElGamalMachine()

pub_key, pri_key = machine.create_key_full(12)
print(pub_key)

print()

print(pri_key)

print()

encrypted = machine.encrypt_full("aufaf20", pub_key)
print(encrypted)

print()

decrypted = machine.decrypt_full(encrypted, pri_key)
print(decrypted)

print()


NjE1LDYxMyAzNjk3IDMgLTIgMTM1NywzNjk2IDI=

MjQzNSAzNjk3IDMgLTIgMg==

ENCODED
(198, 407)
(246, 121)
(208, 1573)
(198, 407)
(208, 1573)
(85, 26)
(80, 532)

kB		:(657, 835)
PmkPb	:(2615, 131)
kB		:(448, 2525)
PmkPb	:(2415, 258)
kB		:(2663, 2571)
PmkPb	:(1101, 2577)
kB		:(2435, 372)
PmkPb	:(3034, 943)
kB		:(2403, 51)
PmkPb	:(1475, 1664)
kB		:(351, 1016)
PmkPb	:(3470, 2660)
kB		:(2563, 2317)
PmkPb	:(1986, 2858)
657,835,2615,131
448,2525,2415,258
2663,2571,1101,2577
2435,372,3034,943
2403,51,1475,1664
351,1016,3470,2660
2563,2317,1986,2858

DECRYPTED
(198, 407)
(246, 121)
(208, 1573)
(198, 407)
(208, 1573)
(85, 26)
(80, 532)

aufaf20



In [258]:
machine = ECCElGamalMachine()

p, curve_a, curve_b, B, k = machine.create_agreement(12)

print()

print("GENERATE RANDOM AGREEMENT")
print("p\t: " + str(p))
print("crv_a\t: " + str(curve_a))
print("crv_b\t: " + str(curve_b))
print("B\t: " + str(B))
print("k\t: " + str(k))

print()

alice_public_key, alice_private_key = machine.create_key(p, curve_a, curve_b, B)
print("GENERATE ALICE KEYS")
print("public_key\t: " + str(alice_public_key))
print("private_key\t: " + str(alice_private_key))

print()

bob_public_key, bob_private_key = machine.create_key(p, curve_a, curve_b, B)
print("GENERATE BOB KEYS")
print("public_key\t: " + str(bob_public_key))
print("private_key\t: " + str(bob_private_key))

print()

print("ENCRYPTED")
encrypted = machine.encrypt("AUFA Fadhlurohman", bob_public_key, p, curve_a, curve_b, B, k)
print(encrypted)

print()

print("DECODED")
decrypted = machine.decrypt(encrypted, bob_private_key, p, curve_a, curve_b, k)
print(decrypted)



GENERATE RANDOM AGREEMENT
p	: 3331
crv_a	: -8
crv_b	: -110
B	: (2019, 2882)
k	: 2

GENERATE ALICE KEYS
public_key	: (2335, 1564)
private_key	: 1339

GENERATE BOB KEYS
public_key	: (743, 327)
private_key	: 213

ENCRYPTED
ENCODED
(130, 1395)
(169, 1432)
(136, 556)
(130, 1395)
(73, 1248)
(136, 556)
(188, 831)
(195, 859)
(200, 317)
(211, 1260)
(233, 1064)
(222, 356)
(215, 1459)
(200, 317)
(212, 1116)
(188, 831)
(213, 662)

kB		:(402, 1279)
PmkPb	:(331, 1896)
kB		:(841, 1351)
PmkPb	:(296, 1742)
kB		:(476, 92)
PmkPb	:(316, 2101)
kB		:(3046, 1306)
PmkPb	:(162, 2209)
kB		:(1816, 2467)
PmkPb	:(1585, 535)
kB		:(2384, 891)
PmkPb	:(3032, 2606)
kB		:(3041, 2489)
PmkPb	:(1269, 2676)
kB		:(173, 879)
PmkPb	:(2440, 2262)
kB		:(833, 3024)
PmkPb	:(122, 2772)
kB		:(1813, 1686)
PmkPb	:(2536, 874)
kB		:(2223, 357)
PmkPb	:(184, 3171)
kB		:(2436, 913)
PmkPb	:(2951, 3117)
kB		:(2467, 206)
PmkPb	:(2974, 715)
kB		:(2022, 2710)
PmkPb	:(2528, 3039)
kB		:(758, 2189)
PmkPb	:(1657, 614)
kB		:(308, 461)
PmkPb	:(1501,

In [ ]:
# ALICE
# Kunci Privat = a
# Kunci Publik = PA = a . B

# BOB
# Kunci Privat = b
# Kunci Publik = PB = b . B

# Bilangan Acak k dalam interval [1, p-1]

# Ciphertext(PM) = PC = [(kB), (PM + kPB)]
# b . (kB) 
